利用tf-idf计算文本相似度

词频TF，词频是一个词语在文章或句子中出现的次数。如果一个词很重要，很明显是应该在一个文章中出现很多次的，但是这也不是绝对的，比如“地”，“的”，“啊”等词，它们出现的次数对一篇文章的中心思想没有一点帮助，只是中文语法结构的一部分而已。这类词也被称为“停用词”。所以，在计算一篇文章的词频时，停用词是应该过滤掉的。
TF=某个词在文档中出现的次数/该文档出现最多的词的出现次数

如果某个词比较少见（在我们准备的文章库中的占比比较少），但是它在这篇文章中多次出现，那么它很可能反映了这篇文章的特性，正是我们所需要的关键词。在此，在词频TF的基础上又引出了反文档频率IDF的概念。一般来说，在一篇文章或一个句子来说，对于每个词都有不同的重要性，这也就是词的权重。在词频的基础上，赋予每一个词的权重，进一步体现该词的重要性。比如一篇报道中国农业养殖的新闻报道。最常见的词（“的”、“是”、“在”）给予最小的权重，较常见的词（“国内”、“中国”、“报道”）给予较小的权重，较少见的词（“养殖”、“维基”）。所以刻画能力强的词语，权重应该是最高的。
LDF=log(语料库文章总数/包含改词的文档数 + 1)

将TF和IDF进行相乘，就得到了一个词的TF-IDF值，某个词对文章重要性越高，该值越大，于是排在前面的几个词，就是这篇文章的关键词。（在实际中，还要考虑词的词性等多维度的特性，动词，名词，形容词的刻画能力也是有所差别的；因社会热点而词的刻画性爆发式提高(比如 打call)）。

In [92]:
import jieba
from gensim import corpora,models,similarities
from collections import defaultdict
doc0 = "我不喜欢上海"
doc1 = "上海是一个好地方"
doc2 = "北京是一个好地方"
doc3 = "上海好吃的在哪里"
doc4 = "上海好玩的在哪里"
doc5 = "我喜欢上海的小吃"
doc6 = "上海路和上海人"
doc7 = "喜欢小吃"
doc_test="我喜欢上海的小吃"

all_doc = []
all_doc.append(doc0)
all_doc.append(doc1)
all_doc.append(doc2)
all_doc.append(doc3)
all_doc.append(doc4)
all_doc.append(doc5)
all_doc.append(doc6)
all_doc.append(doc7)

all_doc_list = []
for doc in all_doc:
    doc_list = [word for word in jieba.cut(doc)]
    all_doc_list.append(doc_list)
print(all_doc_list)

[['我', '不', '喜欢', '上海'], ['上海', '是', '一个', '好', '地方'], ['北京', '是', '一个', '好', '地方'], ['上海', '好吃', '的', '在', '哪里'], ['上海', '好玩', '的', '在', '哪里'], ['我', '喜欢', '上海', '的', '小吃'], ['上海', '路', '和', '上海', '人'], ['喜欢', '小吃']]


In [93]:
# 用dictionary方法获取词袋（bag-of-words)
dictionary = corpora.Dictionary(all_doc_list)

In [94]:
# dictionary的一些用法：
print(dictionary.dfs) #字典，{单词id，在多少文档中出现}
print(dictionary.num_docs)#文档数目
print(dictionary.num_pos)#所有词的个数
for key in dictionary.items():
    print(key)#(17183, '龙骨随葬') 单词id:单词
print(dictionary.num_nnz) #每个文件中不重复词个数的和

{3: 2, 1: 1, 2: 3, 0: 6, 7: 2, 4: 2, 6: 2, 5: 2, 8: 1, 11: 1, 12: 3, 10: 2, 9: 2, 13: 1, 14: 2, 17: 1, 16: 1, 15: 1}
8
36
(0, '上海')
(1, '不')
(2, '喜欢')
(3, '我')
(4, '一个')
(5, '地方')
(6, '好')
(7, '是')
(8, '北京')
(9, '哪里')
(10, '在')
(11, '好吃')
(12, '的')
(13, '好玩')
(14, '小吃')
(15, '人')
(16, '和')
(17, '路')
35


In [95]:
#对稀疏向量进行进一步处理，得到新的语料库
corpus=[dictionary.doc2bow(doc) for doc  in all_doc_list]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)], [(0, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(0, 1), (9, 1), (10, 1), (12, 1), (13, 1)], [(0, 1), (2, 1), (3, 1), (12, 1), (14, 1)], [(0, 2), (15, 1), (16, 1), (17, 1)], [(2, 1), (14, 1)]]


In [96]:
doc_test_list = [word for word in jieba.cut(doc_test)]
doc_test_vec = dictionary.doc2bow(doc_test_list)

In [97]:
#将新的语料库通过tfidfmodel进行处理，得到tfidf
#完成对corpus中出现的每一个特征的IDF值的统计工作即词语普遍重要性的度量,返回一个权重
tfidf=models.TfidfModel(corpus)


In [98]:
#得到特征数
#dictionary.token2id:字典，{词，对应的单词id}  dictionary.token2id.keys():单词个数
featureNum=len(dictionary.token2id.keys())
# #计算稀疏矩阵的相似性,稀疏矩阵相似度，从而建立索引，通过tfidf[corpus]和特征对应起来，
# 则可直接找到相应的权重（相似度），也就是建立了索引

In [99]:
index=similarities.SparseMatrixSimilarity(tfidf[corpus],num_features=featureNum)
# #得到最终相似度结果
print(tfidf[doc_test_vec])
sim=index[tfidf[doc_test_vec]]
print(sim)

[(0, 0.11893745470544932), (2, 0.4055078366880686), (3, 0.5731407639652386), (12, 0.4055078366880686), (14, 0.5731407639652386)]
[0.4542352  0.01227498 0.         0.14231318 0.14231318 0.99999994
 0.01876213 0.7020875 ]


In [100]:
sorted(enumerate(sim), key=lambda item: -item[1])

[(5, 0.99999994),
 (7, 0.7020875),
 (0, 0.4542352),
 (3, 0.14231318),
 (4, 0.14231318),
 (6, 0.01876213),
 (1, 0.012274977),
 (2, 0.0)]

参考： https://www.cxybb.com/article/weixin_43758551/113918690